<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/adapters_pretraining_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This tests using an adapter for pretraining

# Setup

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/AdaptOrDie'

Mounted at /content/drive
/content/drive/MyDrive/AdaptOrDie


In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [ ]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [ ]:
from datasets import load_dataset
dataset_name = "BigTMiami/amazon_25M_10_000_condensed"
dataset = load_dataset(dataset_name)
print(dataset)
print_gpu_utilization()

Generating train split:   0%|          | 0/1718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/866 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 866
    })
})
GPU 448 MB


In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU 448 MB


In [ ]:
adapter_hub_name = "domain_adapter_test"
adapter_name = "domain_training_test"
adapter_type = "seq_bn" # could be "lora", etc.

# Add a new adapter
model.add_adapter(adapter_name, config=adapter_type)

# Add head for masked language modeling
model.add_masked_lm_head(adapter_name)

# Set the adapter to be used for training
model.train_adapter(adapter_name)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
block_size = tokenizer.model_max_length
print(f"block_size:{block_size}")
print_gpu_utilization()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

block_size:512
GPU 448 MB


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

# Train

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="adapter_training_output",
    learning_rate=1e-4,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=200,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [ ]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
)
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


GPU 1403 MB


In [ ]:
eval_results = trainer.evaluate()
non_trained_eval_loss = eval_results["eval_loss"]
print_gpu_utilization()
print(f"Non Trained Eval Loss: {non_trained_eval_loss:.2f}")
print(eval_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


GPU 3435 MB
Non Trained Eval Loss: 16.77
{'eval_loss': 16.76521873474121, 'eval_runtime': 11.4299, 'eval_samples_per_second': 75.766, 'eval_steps_per_second': 9.536}


In [ ]:
results = trainer.train()
print_gpu_utilization()
print(results)

Step,Training Loss
200,6.231700


GPU 8745 MB
TrainOutput(global_step=215, training_loss=6.126039549361828, metrics={'train_runtime': 43.9627, 'train_samples_per_second': 39.079, 'train_steps_per_second': 4.891, 'total_flos': 463518432792576.0, 'train_loss': 6.126039549361828, 'epoch': 1.0})


In [ ]:
eval_results = trainer.evaluate()
trained_eval_loss = eval_results["eval_loss"]
print_gpu_utilization()
print(f"Trained Eval Loss: {trained_eval_loss:.2f}")
print(eval_results)

GPU 8745 MB
Trained Eval Loss: 4.40
{'eval_loss': 4.3955535888671875, 'eval_runtime': 10.6494, 'eval_samples_per_second': 81.319, 'eval_steps_per_second': 10.235, 'epoch': 1.0}


In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name
)

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/domain_adapter_test/commit/d101cb6cd5f69e8c293069db51c3668c1881274a', commit_message='Upload model', commit_description='', oid='d101cb6cd5f69e8c293069db51c3668c1881274a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()